In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, StoppingCriteria, StoppingCriteriaList

class StopTokenCriteria(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        # 检查最新生成的 token 是否是停止标记
        return input_ids[0, -1] in self.stop_token_id

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("/home/dozhang/nlcmt1/HuggingfaceModels/Meta-Llama-3.1-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("/home/dozhang/nlcmt1/HuggingfaceModels/Meta-Llama-3.1-8B-Instruct").to(device)

[2024-08-12 09:00:24,867] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/home/dozhang/miniconda3/envs/py31/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [42]:
# Encode input text
input_text = "<|start_header_id|>system<|end_header_id|>\n\nYou are a helpful AI assistant for travel tips and recommendations<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nKylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

input_text = "<|eot_id|>Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?"
input_text = "<|eot_id|>Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?"

inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Create attention mask
attention_mask = inputs.attention_mask

# 定义停止条件
stop_token_id = tokenizer.convert_tokens_to_ids(["</s>", "<|eot_id|>", "<|end_of_text|>", "<|end_header_id|>", "<|start_header_id|>"])
stopping_criteria = StoppingCriteriaList([StopTokenCriteria(stop_token_id)])

#import pdb; pdb.set_trace()
# Generate text
outputs = model.generate(inputs["input_ids"], attention_mask=attention_mask, max_length=2048, num_return_sequences=1, 
                         pad_token_id=tokenizer.eos_token_id, stopping_criteria=stopping_criteria)

# Decode the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print (input_text)
print (">>>>")
print(generated_text)

<|eot_id|>Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them?
>>>>
Kylar went to the store to buy glasses for his new apartment. One glass costs $5, but every second glass costs only 60% of the price. Kylar wants to buy 16 glasses. How much does he need to pay for them? (Round your answer to the nearest cent)
## Step 1: Determine the cost of the first glass
The cost of the first glass is $5.

## Step 2: Determine the cost of the second glass
The second glass costs 60% of $5. To find the cost, multiply 0.6 by $5.

## Step 3: Calculate the cost of the second glass
0.6 * $5 = $3.

## Step 4: Determine the pattern for the cost of subsequent glasses
Every second glass costs $3, and the rest cost $5.

## Step 5: Calculate the number of glasses that cost $5 and the number of glasses that cost $3
Since there are 16 glasses in total, and every s